In [1]:
# here comes the imports!

import numpy as np
import pandas as pd
import seaborn as sns


import matplotlib.pyplot as plt;
%matplotlib inline

import datetime
import time

In [2]:
# imports relativos a machine learning
import sklearn
from sklearn.model_selection import train_test_split

import h2o
from h2o.automl import H2OAutoML


from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.metrics import roc_auc_score


# Versões dos pacotes utilizados!
## Isto é relevante com respeito às questões 3 e 4, em que é pedido para especificar as versões das bibliotecas de Python para que os resultados possam ser reproduzidos.


In [3]:


print('A versão do numpy é {}'.format(np.__version__))
print('A versão do pandas é {}'.format(pd.__version__))
print('A versão do seaborn é {}'.format(sns.__version__))
print('A versão do sklearn é {}'.format(sklearn.__version__))
print('A versão do h2o é {}'.format(h2o.__version__))



A versão do numpy é 1.16.2
A versão do pandas é 0.23.4
A versão do seaborn é 0.9.0
A versão do sklearn é 0.20.3
A versão do h2o é 3.18.0.2


# vamos ler o dataset utilizando o pandas!

In [4]:
data_json='./dataset.json'

df=pd.read_json(data_json)

In [5]:
df

,branch_id,customer_code,group_code,is_churn,item_code,item_total_price,order_id,quantity,register_date,sales_channel,segment_code,seller_code,total_price,unit_price
0,0,143,0,0.0,854,292.91,21804,10,2017-11-10T00:00:00Z,0,0,190,1613.53,25.04
1,0,433,0,0.0,246,287.19,5486,20,2011-05-16T00:00:00Z,1,5,153,11163.69,12.33
2,0,486,0,0.0,1420,184.84,22662,12,2018-01-24T00:00:00Z,0,0,166,6432.12,12.80
3,0,107,0,0.0,1963,189.18,3956,18,2010-07-28T00:00:00Z,1,0,156,831.82,10.51
4,0,768,0,0.0,1786,66.87,4730,5,2010-12-17T00:00:00Z,1,0,218,1736.48,11.82
5,0,740,0,0.0,2854,226.80,1835,10,2009-05-07T00:00:00Z,8,0,213,445.10,21.60
6,0,58,0,0.0,246,105.51,11450,6,2013-12-17T00:00:00Z,0,0,179,573.06,15.68
7,0,408,0,0.0,1842,34.26,2504,6,2009-09-29T00:00:00Z,1,0,199,1201.44,5.71
8,0,25,0,0.0,1767,482.60,21773,10,2017-11-09T00:00:00Z,0,0,94,1397.78,48.26
9,0,58,0,0.0,410,66.42,843,2,2008-08-19T00:00:00Z,1,0,179,2687.76,31.63


In [7]:
# verificando o tipo das variaveis
df.dtypes


branch_id             int64
customer_code         int64
group_code            int64
is_churn            float64
item_code             int64
item_total_price    float64
order_id              int64
quantity              int64
register_date        object
sales_channel         int64
segment_code          int64
seller_code           int64
total_price         float64
unit_price          float64
dtype: object

## Os tipos das variaveis parecem bem razoaveis
## A coluna 'register_date' que é do tipo objeto, ela representa datas

# Vamos ver as dimensões desse DataFrame!
# Podemos acessar isso através do atributo .shape

In [8]:
df.shape

(204428, 14)

In [9]:
print('Temos {0} linhas.'.format(df.shape[0]))
print('Temos {0} colunas.'.format(df.shape[1]))

Temos 204428 linhas.
Temos 14 colunas.


# Verificando se há duplicatas!


In [10]:

print(df.duplicated().any())
print(df.duplicated().sum())

True
53


# Há 53 duplicatas! Vamos retirá-las.

In [11]:
df.drop_duplicates(inplace=True)

In [12]:
# verificando, novamente, se há duplicatas

print(df.duplicated().any())
print(df.duplicated().sum())

False
0


# Não há mais valores duplicados!

# Vamos ver valores faltantes nesse DataFrame

In [13]:
df.isnull().any()

branch_id           False
customer_code       False
group_code          False
is_churn             True
item_code           False
item_total_price    False
order_id            False
quantity            False
register_date       False
sales_channel       False
segment_code        False
seller_code         False
total_price         False
unit_price          False
dtype: bool

In [14]:
df.isnull().sum()

branch_id              0
customer_code          0
group_code             0
is_churn            1909
item_code              0
item_total_price       0
order_id               0
quantity               0
register_date          0
sales_channel          0
segment_code           0
seller_code            0
total_price            0
unit_price             0
dtype: int64

# só há valores faltantes na coluna 'is_churn'

# Há 1909 valores faltantes na coluna 'is_churn', o que pode parecer bastante, mas o correto é ver como esse número se compara com o total de linhas

In [15]:
print('Porcentagem de valores faltantes {0}%'.format(100*1909/df.shape[0]))

Porcentagem de valores faltantes 0.9340672782874617%


# Logo, temos menos de 1% de valores faltantes!

# Vamos proceder removendo esses valores, que são relativamente poucos, que estão faltando na coluna 'is_churn'

In [16]:
df.dropna(inplace=True)

# Vamos verificar se ainda há valores faltantes

In [17]:
df.isnull().any()

branch_id           False
customer_code       False
group_code          False
is_churn            False
item_code           False
item_total_price    False
order_id            False
quantity            False
register_date       False
sales_channel       False
segment_code        False
seller_code         False
total_price         False
unit_price          False
dtype: bool

# Não há mais valores faltantes!

# Lidando com a coluna que é do tipo objeto, que representa data!

In [18]:
df['register_date']

0         2017-11-10T00:00:00Z
1         2011-05-16T00:00:00Z
2         2018-01-24T00:00:00Z
3         2010-07-28T00:00:00Z
4         2010-12-17T00:00:00Z
5         2009-05-07T00:00:00Z
6         2013-12-17T00:00:00Z
7         2009-09-29T00:00:00Z
8         2017-11-09T00:00:00Z
9         2008-08-19T00:00:00Z
10        2011-01-27T00:00:00Z
11        2013-07-31T00:00:00Z
12        2016-07-19T00:00:00Z
13        2015-02-18T00:00:00Z
14        2017-06-05T00:00:00Z
15        2008-01-18T00:00:00Z
16        2017-07-10T00:00:00Z
17        2018-01-29T00:00:00Z
18        2018-06-13T00:00:00Z
19        2016-03-15T00:00:00Z
20        2016-03-10T00:00:00Z
21        2011-07-09T00:00:00Z
22        2018-03-21T00:00:00Z
23        2016-10-24T00:00:00Z
24        2010-05-07T00:00:00Z
25        2011-09-22T00:00:00Z
26        2008-06-26T00:00:00Z
27        2011-04-29T00:00:00Z
28        2017-02-02T00:00:00Z
29        2013-12-04T00:00:00Z
                  ...         
204398    2014-02-21T00:00:00Z
204399  

In [19]:
# vamos lidar com essa coluna!

# redefinindo os índices

In [21]:
df=df.reset_index(drop=True)

In [22]:
# Usando série temporal!

In [23]:
df3=df.copy()

In [24]:
df3['Date']=pd.to_datetime(df3.register_date)

In [25]:
df3.sort_values(by='Date',inplace=True)

In [26]:
df3.drop('register_date',axis=1,inplace=True)

In [27]:
df3

,branch_id,customer_code,group_code,is_churn,item_code,item_total_price,order_id,quantity,sales_channel,segment_code,seller_code,total_price,unit_price,Date
91229,0,588,0,0.0,1768,166.04,0,2,6,0,212,710.79,79.07,2008-01-04
163387,0,588,0,0.0,2675,255.90,0,20,6,0,212,710.79,11.42,2008-01-04
51446,0,588,0,0.0,282,363.08,0,20,6,0,212,710.79,16.21,2008-01-04
81273,0,114,2,1.0,2675,382.65,1,50,7,2,130,341.65,6.83,2008-01-05
132216,0,435,0,0.0,2624,495.83,2,32,1,0,3,3035.75,14.76,2008-01-07
13263,0,681,0,0.0,1778,116.56,6,3,6,0,45,705.94,34.69,2008-01-07
171484,0,681,0,0.0,416,128.46,6,10,6,0,45,705.94,11.47,2008-01-07
78831,0,613,0,0.0,1767,520.61,4,10,8,0,177,1006.41,49.58,2008-01-07
149043,0,681,0,0.0,282,155.95,6,8,6,0,45,705.94,17.40,2008-01-07
65259,0,681,0,0.0,2630,176.85,6,10,6,0,45,705.94,16.84,2008-01-07


In [28]:
#df3=df3.reset_index(drop=True,inplace=True)
df3=df3.reset_index(drop=True)

In [29]:
df3

,branch_id,customer_code,group_code,is_churn,item_code,item_total_price,order_id,quantity,sales_channel,segment_code,seller_code,total_price,unit_price,Date
0,0,588,0,0.0,1768,166.04,0,2,6,0,212,710.79,79.07,2008-01-04
1,0,588,0,0.0,2675,255.90,0,20,6,0,212,710.79,11.42,2008-01-04
2,0,588,0,0.0,282,363.08,0,20,6,0,212,710.79,16.21,2008-01-04
3,0,114,2,1.0,2675,382.65,1,50,7,2,130,341.65,6.83,2008-01-05
4,0,435,0,0.0,2624,495.83,2,32,1,0,3,3035.75,14.76,2008-01-07
5,0,681,0,0.0,1778,116.56,6,3,6,0,45,705.94,34.69,2008-01-07
6,0,681,0,0.0,416,128.46,6,10,6,0,45,705.94,11.47,2008-01-07
7,0,613,0,0.0,1767,520.61,4,10,8,0,177,1006.41,49.58,2008-01-07
8,0,681,0,0.0,282,155.95,6,8,6,0,45,705.94,17.40,2008-01-07
9,0,681,0,0.0,2630,176.85,6,10,6,0,45,705.94,16.84,2008-01-07


In [30]:
# Extracting date features
#df3['Date'].dt.weekofyear

df3['dayofmonth'] = df3['Date'].dt.day
df3['dayofyear'] = df3['Date'].dt.dayofyear
df3['dayofweek'] = df3['Date'].dt.dayofweek
df3['month'] = df3['Date'].dt.month
df3['year'] = df3['Date'].dt.year
df3['weekofyear'] = df3['Date'].dt.weekofyear
df3['is_month_start'] = (df3['Date'].dt.is_month_start).astype(int)
df3['is_month_end'] = (df3['Date'].dt.is_month_end).astype(int)

In [31]:
df3.drop('Date',axis=1,inplace=True)

In [32]:
df3

,branch_id,customer_code,group_code,is_churn,item_code,item_total_price,order_id,quantity,sales_channel,segment_code,...,total_price,unit_price,dayofmonth,dayofyear,dayofweek,month,year,weekofyear,is_month_start,is_month_end
0,0,588,0,0.0,1768,166.04,0,2,6,0,...,710.79,79.07,4,4,4,1,2008,1,0,0
1,0,588,0,0.0,2675,255.90,0,20,6,0,...,710.79,11.42,4,4,4,1,2008,1,0,0
2,0,588,0,0.0,282,363.08,0,20,6,0,...,710.79,16.21,4,4,4,1,2008,1,0,0
3,0,114,2,1.0,2675,382.65,1,50,7,2,...,341.65,6.83,5,5,5,1,2008,1,0,0
4,0,435,0,0.0,2624,495.83,2,32,1,0,...,3035.75,14.76,7,7,0,1,2008,2,0,0
5,0,681,0,0.0,1778,116.56,6,3,6,0,...,705.94,34.69,7,7,0,1,2008,2,0,0
6,0,681,0,0.0,416,128.46,6,10,6,0,...,705.94,11.47,7,7,0,1,2008,2,0,0
7,0,613,0,0.0,1767,520.61,4,10,8,0,...,1006.41,49.58,7,7,0,1,2008,2,0,0
8,0,681,0,0.0,282,155.95,6,8,6,0,...,705.94,17.40,7,7,0,1,2008,2,0,0
9,0,681,0,0.0,2630,176.85,6,10,6,0,...,705.94,16.84,7,7,0,1,2008,2,0,0


# Agora vamos às questões!

# Question 1 (10 Points)
List as many use cases for the dataset as possible.

Re: Os casos de uso de uso para o conjunto de dados são:

1. saber em função do preço total 'total price' quantos clientes estão comprando certo item, criando intervalos para o preço total onde há maior lucro; 
2. saber qual canal de venda gera maior lucro 'sales_channel', determinando uma preferência e onde investir;
3. determinar a partir de 'register_date' (data de pedido) qual ano período gerou maior lucro;
4. determinar a partir de 'total_price' (preço total) qual faixa gera mais lucro, buscando assim maximizar o lucro e otimizar o tipo de cliente;
5. a partir de 'segment_code' (código de segmento que o cliente pertence) otimizar ...
6. a partir de 'group_code' (código de grupo) otimizar o lucro da empresa e encontrar qual é o grupo ótimo de consumidor em que se deve ser investido mais esforços;
7. criar um modelo preditivo a partir de 'is_churn' para obter informações sobre qual grupo, segmento, canal de venda, vendedor, tem maior chance de que esse cliente retorne;
8. estudar qual 'seller_code' (código de vendedor) tem mais sucesso com os clientes.



# Question 2 (10 Points)
Pick one of the use cases you listed in question 1 and describe how building a statistical model based on the dataset could best be used to improve the business this data comes from.

Re: Escolhendo o caso de uso $7$ da questão $1$ podemos criar um modelo estatístico, de machine learning, para prever se um dado cliente voltará, estudando então a influência dos features na predição dos dados.

# Question 3 (20 Points)
Implement the model you described in question 2, preferably in Python. The code has to retrieve the data, train and test a statistical model, and report relevant performance criteria. Ideally, we should be able to replicate your analysis from your submitted source-code, so please explicit the versions of the tools and packages you are using.

# A versão dos pacotes!

In [33]:


print('A versão do numpy é {}'.format(np.__version__))
print('A versão do pandas é {}'.format(pd.__version__))
print('A versão do seaborn é {}'.format(sns.__version__))
print('A versão do sklearn é {}'.format(sklearn.__version__))
print('A versão do h2o é {}'.format(h2o.__version__))



A versão do numpy é 1.16.2
A versão do pandas é 0.23.4
A versão do seaborn é 0.9.0
A versão do sklearn é 0.20.3
A versão do h2o é 3.18.0.2


# separando em dados de treino e teste

In [34]:
#df_ml=df2.copy()
df_ml=df3.copy()

In [35]:
#msk = np.random.random(len(df_ml)) < 0.8
# criando a mascara
percent=0.66

msk=[]
for i in range(len(df_ml)):
    msk.append(i<=percent*len(df_ml))
msk=np.array(msk)

In [36]:
train = df_ml[msk]

In [37]:
test=df_ml[~msk]

# Vamos utilizar h2o para automatizar os cálculos estatísticos de machine learning.
# Vamos initializar h2o!

In [40]:
h2o.init()


Checking whether there is an H2O instance running at http://localhost:54321. connected.


H2O cluster uptime:,11 secs
H2O cluster timezone:,America/Sao_Paulo
H2O data parsing timezone:,UTC
H2O cluster version:,3.18.0.2
H2O cluster version age:,"1 year, 5 months and 6 days !!!"
H2O cluster name:,H2O_from_python_vagner_ig2b41
H2O cluster total nodes:,1
H2O cluster free memory:,3.433 Gb
H2O cluster total cores:,8
H2O cluster allowed cores:,8
H2O cluster status:,"locked, healthy"


In [41]:
train.iloc[-1]

branch_id               0.00
customer_code         377.00
group_code              0.00
is_churn                0.00
item_code            1048.00
item_total_price       97.08
order_id            16241.00
quantity                6.00
sales_channel           0.00
segment_code            0.00
seller_code            50.00
total_price          3216.77
unit_price             16.18
dayofmonth             17.00
dayofyear              48.00
dayofweek               2.00
month                   2.00
year                 2016.00
weekofyear              7.00
is_month_start          0.00
is_month_end            0.00
Name: 133627, dtype: float64

In [42]:
test.iloc[1]

branch_id               0.00
customer_code         428.00
group_code              0.00
is_churn                0.00
item_code             410.00
item_total_price      138.60
order_id            16245.00
quantity                3.00
sales_channel           0.00
segment_code            0.00
seller_code            39.00
total_price          1697.39
unit_price             45.64
dayofmonth             17.00
dayofyear              48.00
dayofweek               2.00
month                   2.00
year                 2016.00
weekofyear              7.00
is_month_start          0.00
is_month_end            0.00
Name: 133629, dtype: float64

In [43]:
train=h2o.H2OFrame(train)


/home/vagner/anaconda3/lib/python3.6/site-packages/h2o/utils/shared_utils.py:170: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  data = _handle_python_lists(python_obj.as_matrix().tolist(), -1)[1]


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [44]:
test=h2o.H2OFrame(test)


/home/vagner/anaconda3/lib/python3.6/site-packages/h2o/utils/shared_utils.py:170: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  data = _handle_python_lists(python_obj.as_matrix().tolist(), -1)[1]


Parse progress: |█████████████████████████████████████████████████████████| 100%


In [45]:
# Identify predictors and response
x = train.columns
y = "is_churn"
x.remove(y)


In [46]:
x

['branch_id',
 'customer_code',
 'group_code',
 'item_code',
 'item_total_price',
 'order_id',
 'quantity',
 'sales_channel',
 'segment_code',
 'seller_code',
 'total_price',
 'unit_price',
 'dayofmonth',
 'dayofyear',
 'dayofweek',
 'month',
 'year',
 'weekofyear',
 'is_month_start',
 'is_month_end']

# Let us visualize train

In [47]:
train

branch_id,customer_code,group_code,is_churn,item_code,item_total_price,order_id,quantity,sales_channel,segment_code,seller_code,total_price,unit_price,dayofmonth,dayofyear,dayofweek,month,year,weekofyear,is_month_start,is_month_end
0,588,0,0,1768,166.04,0,2,6,0,212,710.79,79.07,4,4,4,1,2008,1,0,0
0,588,0,0,2675,255.9,0,20,6,0,212,710.79,11.42,4,4,4,1,2008,1,0,0
0,588,0,0,282,363.08,0,20,6,0,212,710.79,16.21,4,4,4,1,2008,1,0,0
0,114,2,1,2675,382.65,1,50,7,2,130,341.65,6.83,5,5,5,1,2008,1,0,0
0,435,0,0,2624,495.83,2,32,1,0,3,3035.75,14.76,7,7,0,1,2008,2,0,0
0,681,0,0,1778,116.56,6,3,6,0,45,705.94,34.69,7,7,0,1,2008,2,0,0
0,681,0,0,416,128.46,6,10,6,0,45,705.94,11.47,7,7,0,1,2008,2,0,0
0,613,0,0,1767,520.61,4,10,8,0,177,1006.41,49.58,7,7,0,1,2008,2,0,0
0,681,0,0,282,155.95,6,8,6,0,45,705.94,17.4,7,7,0,1,2008,2,0,0
0,681,0,0,2630,176.85,6,10,6,0,45,705.94,16.84,7,7,0,1,2008,2,0,0


# Let us visualize the test

In [48]:
test

branch_id,customer_code,group_code,is_churn,item_code,item_total_price,order_id,quantity,sales_channel,segment_code,seller_code,total_price,unit_price,dayofmonth,dayofyear,dayofweek,month,year,weekofyear,is_month_start,is_month_end
0,237,0,0,1767,542.66,16232,10,6,0,288,2341.66,47.65,17,48,2,2,2016,7,0,0
0,428,0,0,410,138.6,16245,3,0,0,39,1697.39,45.64,17,48,2,2,2016,7,0,0
0,812,0,0,1760,14.4,16240,10,0,0,3,2361.59,1.44,17,48,2,2,2016,7,0,0
0,636,0,0,204,117.08,16233,6,0,0,177,2702.81,19.18,17,48,2,2,2016,7,0,0
0,103,0,0,361,295.08,16230,10,0,0,198,5736.58,25.91,17,48,2,2,2016,7,0,0
0,636,0,0,1788,180.6,16233,4,0,0,177,2702.81,44.38,17,48,2,2,2016,7,0,0
0,636,0,0,2630,183.32,16233,10,0,0,177,2702.81,18.11,17,48,2,2,2016,7,0,0
0,237,0,0,1767,542.66,16229,10,6,0,288,2174.14,47.65,17,48,2,2,2016,7,0,0
0,377,0,0,1045,149.34,16241,6,0,0,50,3216.77,24.89,17,48,2,2,2016,7,0,0
0,812,0,0,1434,100.3,16240,10,0,0,3,2361.59,10.03,17,48,2,2,2016,7,0,0


In [49]:
# For binary classification, response should be a factor
train[y] = train[y].asfactor()


In [50]:
test[y] = test[y].asfactor()


In [51]:
# Vamos limitar o número máximo de modelos
# Vamos tirar cross-validation, colocando nfols=0
# vamos limitar o tempo maximo de calculo

In [52]:
# Run AutoML for 20 base models (limited to 1 hour max runtime by default)
aml = H2OAutoML(max_models=5,nfolds=0,max_runtime_secs=1200, seed=1)
aml.train(x=x, y=y, training_frame=train)


AutoML progress: |████████████████████████████████████████████████████████| 100%
Parse progress: |█████████████████████████████████████████████████████████| 100%


In [53]:
#salvando o modelo

In [54]:
#model_path = h2o.save_model(model=aml, path="./mymodel", force=True)
#h2o.save_model(aml.leader, path = "./mymodel")


In [55]:
aml.leader

Model Details
H2OGradientBoostingEstimator :  Gradient Boosting Machine
Model Key:  GBM_grid_0_AutoML_20190811_201902_model_0


ModelMetricsBinomial: gbm
** Reported on train data. **

MSE: 3.285506325756697e-35
RMSE: 5.731933640366658e-18
LogLoss: 1.484852297690794e-19
Mean Per-Class Error: 0.0
AUC: 1.0
Gini: 1.0
Confusion Matrix (Act/Pred) for max f1 @ threshold = 1.0: 


,0,1,Error,Rate
0,86203.0,0.0,0.0,(0.0/86203.0)
1,0.0,20718.0,0.0,(0.0/20718.0)
Total,86203.0,20718.0,0.0,(0.0/106921.0)


Maximum Metrics: Maximum metrics at their respective thresholds



metric,threshold,value,idx
max f1,1.0,1.0,0.0
max f2,1.0,1.0,0.0
max f0point5,1.0,1.0,0.0
max accuracy,1.0,1.0,0.0
max precision,1.0,1.0,0.0
max recall,1.0,1.0,0.0
max specificity,1.0,1.0,0.0
max absolute_mcc,1.0,1.0,0.0
max min_per_class_accuracy,1.0,1.0,0.0
max mean_per_class_accuracy,1.0,1.0,0.0


Gains/Lift Table: Avg response rate: 19,38 %



...

,group,cumulative_data_fraction,lower_threshold,lift,cumulative_lift,response_rate,cumulative_response_rate,capture_rate,cumulative_capture_rate,gain,cumulative_gain
,1,0.2014485,1.0,4.9257080,4.9257080,1.0,1.0,0.9922766,0.9922766,392.5707981,392.5707981
,2,1.0,1e-19,0.0096718,1.0,0.0019635,0.2030165,0.0077234,1.0,-99.0328203,0.0



Scoring History: 


,timestamp,duration,number_of_trees,training_rmse,training_logloss,training_auc,training_lift,training_classification_error,validation_rmse,validation_logloss,validation_auc,validation_lift,validation_classification_error
,2019-08-11 20:19:20,0.107 sec,0.0,0.3952502,0.4916425,0.5,1.0,0.8062308,0.4023510,0.5048263,0.5,1.0,0.7969835
,2019-08-11 20:19:21,0.613 sec,5.0,0.2582622,0.2570098,0.9981393,5.1607781,0.0094275,0.2603461,0.2599045,0.9981211,4.9257080,0.0087359
,2019-08-11 20:19:21,0.823 sec,10.0,0.1694852,0.1503866,0.9999752,5.1607781,0.0015806,0.1690458,0.1510015,0.9999797,4.9257080,0.0011200
,2019-08-11 20:19:21,1.065 sec,15.0,0.1261993,0.1020219,0.9999780,5.1607781,0.0017770,0.1240061,0.1016378,0.9999821,4.9257080,0.0011947
,2019-08-11 20:19:22,1.963 sec,20.0,0.0932878,0.0674424,0.9999807,5.1607781,0.0015151,0.0900864,0.0666675,0.9999912,4.9257080,0.0006720
---,---,---,---,---,---,---,---,---,---,---,---,---,---
,2019-08-11 20:20:27,1 min 6.427 sec,1015.0,0.0000000,0.0000000,1.0,5.1607781,0.0,0.0000000,0.0000000,1.0,4.9257080,0.0
,2019-08-11 20:20:27,1 min 6.570 sec,1020.0,0.0000000,0.0000000,1.0,5.1607781,0.0,0.0000000,0.0000000,1.0,4.9257080,0.0
,2019-08-11 20:20:27,1 min 6.693 sec,1025.0,0.0000000,0.0000000,1.0,5.1607781,0.0,0.0000000,0.0000000,1.0,4.9257080,0.0
,2019-08-11 20:20:27,1 min 6.827 sec,1030.0,0.0000000,0.0000000,1.0,5.1607781,0.0,0.0000000,0.0000000,1.0,4.9257080,0.0



See the whole table with table.as_data_frame()
Variable Importances: 


variable,relative_importance,scaled_importance,percentage
group_code,38384.4804688,1.0,0.4924955
segment_code,13488.0302734,0.3513928,0.1730594
customer_code,9526.8593750,0.2481956,0.1222352
seller_code,9247.7812500,0.2409250,0.1186545
total_price,3124.4375,0.0813985,0.0400884
sales_channel,2719.0109863,0.0708362,0.0348865
order_id,996.4774170,0.0259604,0.0127854
unit_price,177.9557037,0.0046361,0.0022833
item_code,124.0592651,0.0032320,0.0015918
dayofmonth,51.0453949,0.0013298,0.0006549


In [56]:
aml.leaderboard

model_id,auc,logloss
GBM_grid_0_AutoML_20190811_201902_model_0,"1,000000","0,000000"
GBM_grid_0_AutoML_20190811_201902_model_1,"1,000000","0,000000"
DRF_0_AutoML_20190811_201902,"1,000000","0,005682"
XRT_0_AutoML_20190811_201902,"0,999999","0,018917"
GLM_grid_0_AutoML_20190811_201902_model_0,"0,945772","0,174354"


In [57]:
# If you need to generate predictions on a test set, you can make
# predictions directly on the `"H2OAutoML"` object, or on the leader
# model object directly

preds = aml.predict(test)


Parse progress: |█████████████████████████████████████████████████████████| 100%
gbm prediction progress: |████████████████████████████████████████████████| 100%


In [58]:
(train.as_data_frame)

branch_id,customer_code,group_code,is_churn,item_code,item_total_price,order_id,quantity,sales_channel,segment_code,seller_code,total_price,unit_price,dayofmonth,dayofyear,dayofweek,month,year,weekofyear,is_month_start,is_month_end
0,588,0,0,1768,166.04,0,2,6,0,212,710.79,79.07,4,4,4,1,2008,1,0,0
0,588,0,0,2675,255.9,0,20,6,0,212,710.79,11.42,4,4,4,1,2008,1,0,0
0,588,0,0,282,363.08,0,20,6,0,212,710.79,16.21,4,4,4,1,2008,1,0,0
0,114,2,1,2675,382.65,1,50,7,2,130,341.65,6.83,5,5,5,1,2008,1,0,0
0,435,0,0,2624,495.83,2,32,1,0,3,3035.75,14.76,7,7,0,1,2008,2,0,0
0,681,0,0,1778,116.56,6,3,6,0,45,705.94,34.69,7,7,0,1,2008,2,0,0
0,681,0,0,416,128.46,6,10,6,0,45,705.94,11.47,7,7,0,1,2008,2,0,0
0,613,0,0,1767,520.61,4,10,8,0,177,1006.41,49.58,7,7,0,1,2008,2,0,0
0,681,0,0,282,155.95,6,8,6,0,45,705.94,17.4,7,7,0,1,2008,2,0,0
0,681,0,0,2630,176.85,6,10,6,0,45,705.94,16.84,7,7,0,1,2008,2,0,0


<bound method H2OFrame.as_data_frame of >

In [59]:
lb=aml.leaderboard
lb.head(rows=lb.nrows)

model_id,auc,logloss
GBM_grid_0_AutoML_20190811_201902_model_0,"1,000000","0,000000"
GBM_grid_0_AutoML_20190811_201902_model_1,"1,000000","0,000000"
DRF_0_AutoML_20190811_201902,"1,000000","0,005682"
XRT_0_AutoML_20190811_201902,"0,999999","0,018917"
GLM_grid_0_AutoML_20190811_201902_model_0,"0,945772","0,174354"


In [60]:
print('Generate Predictions!')

testy=aml.leader.predict(test)
#testy=testy.as_data_frame()

Generate Predictions!
gbm prediction progress: |████████████████████████████████████████████████| 100%


In [61]:
# very nice
testy


predict,p0,p1
0,1,1e-19
0,1,1e-19
0,1,1e-19
0,1,1e-19
0,1,1e-19
0,1,1e-19
0,1,1e-19
0,1,1e-19
0,1,3.51619e-19
0,1,1e-19


In [405]:
#(test['is_churn']==testy['predict']).sum()/testy.shape[0]

In [62]:
test['is_churn']

is_churn
0
0
0
0
0
0
0
0
0
0


In [63]:
testy['predict']

predict
0
0
0
0
0
0
0
0
0
0


# Calculation of some metrics of the predicted model!

# The f1_score

In [65]:
f1_score(testy['predict'].as_data_frame(),test['is_churn'].as_data_frame())

0.762225147481839

# The Recall

In [66]:
recall_score(testy['predict'].as_data_frame(),test['is_churn'].as_data_frame())

1.0

# The Precision

In [67]:
precision_score(testy['predict'].as_data_frame(),test['is_churn'].as_data_frame())

0.6158027414526548

# The ROC AUC score

In [68]:
roc_auc_score(testy['predict'].as_data_frame(),test['is_churn'].as_data_frame())

0.9600383474541551

# O valor de 'ROC AUC Score' acima, que é próximo a 1, mostra que temos um modelo bom para classificação binária do target 'is_churn'.

# Question 4 (60 Points)
A. Explain each and every of your design choices, you can use jupyter notebooks. (e.g., preprocessing, model selection, hyper parameters, evaluation criteria). Compare and contrast your choices with alternative methodologies.

## Foi escolhido utilizar o h2o para efetuar os cálculos de machine learning, pois é um pacote que otimiza o tempo de cálculo de cada algoritmo e tem uma ótima automatização. O h2o consegue achar o melhor modelo e qual modelo prediz melhor o 'target', juntamente com os parâmetros do modelo que vão gerar os melhores resultados, permitindo saber qual a melhor faixa de parâmetros a serem escolhidos no modelo.

## Foi retirada a validação cruzada (cross-validation), colocando o parâmetro nfolds=0, uma vez que ela não é uma boa prática para séries temporais. A melhor técnica no caso para cross validation é a 'walkforward' [https://medium.com/@samuel.monnier/cross-validation-tools-for-time-series-ffa1a5a09bf9]. No entanto, essa técnica de cross validation não está implementada no automl. 

## A métrica mais interessante para o problema de prever 'is_churn', uma vez que é uma classificação binária, é a "ROC AUC" (Area Under the ROC Curve), pois ela está intimamente ligada com a taxa de true positives e falsos positivos.
## O melhor modelo foi obtido com base na métrica "ROC AUC", que é feito automaticamente no h2o. 

## Alternativamente, é interessante olhar para as métricas recall e precision, uma vez que elas estão relacionadas com 'true positives' e 'falsos positivos'. Essas métricas são importantes que elas identificam bem casos em que houve uma predição errada [https://medium.com/@yashwant140393/the-3-pillars-of-binary-classification-accuracy-precision-recall-d2da3d09f664].

B. Describe how you would improve the model in Question 3 if you had more time.

# Seguem a seguir algumas melhorias que poderiam ser feitas no modelo (caso houvesse mais tempo):

1. O modelo poderia ser melhorado usando **feature engineering**, por exemplo, definindo novos features em termos dos antigos.
2. Também poderia ser trabalhada a **remoção de outliers**, removendo, por exemplo, 10% dos dados mais "fora da curva" que seriam os que estão mais distantes da média, com base em uma função custo que é a diferença entre predição (através de uma regressão linear dos dados) e o valor do ponto, e assim eliminando-os do conjunto de dados;
3. Também é possível aumentar o número de modelos testados e o tempo de execução total dos cálculos ou o tempo máximo de cada modelo;
4. Poderíamos também refinar o melhor modelo que foi encontrado (aqui LightGBM) e refinar os parâmetros que o h2o devolve ao final dos cálculos;
5. poderíamos também melhor as features temporais e adicionar feriados regionais e nacionais.